In [1]:
import ee
import geemap
import os
import geopandas as gpd
ee.Initialize()

home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"

In [10]:
# get the YMD from each image in a collection as a list 
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

# forest cover mask
def LC(image):
    landcover = image.clip(roi)
    return landcover.eq(4)

In [19]:
# The LCMAP dataset is available through the GEE community assets 
# It is available from 1985 - 2020 
# Based on that, I want to find which pixels are forested the entire time
roi = ee.FeatureCollection('users/kmcquil/blue_ridge_plus_reference_xl').geometry()
dataset = ee.ImageCollection("projects/sat-io/open-datasets/LCMAP/LCPRI")
dates = ymdList(dataset)

In [24]:
print(dates)

['19850101', '19860101', '19870101', '19880101', '19890101', '19900101', '19910101', '19920101', '19930101', '19940101', '19950101', '19960101', '19970101', '19980101', '19990101', '20000101', '20010101', '20020101', '20030101', '20040101', '20050101', '20060101', '20070101', '20080101', '20090101', '20100101', '20110101', '20120101', '20130101', '20140101', '20150101', '20160101', '20170101', '20180101', '20190101', '20200101']


In [34]:
forest = dataset.map(LC)
forest_sum = forest.sum()

forest_sum_float = forest_sum.float()

In [21]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [23]:
vizParams = {'min': 0, 'max': 35, 'palette': ['white', 'yellow', 'green', 'blue'],}
Map.addLayer(forest_sum, vizParams, 'forest_cover')


In [35]:
task_config = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'lcmap_permanent_forest',
        'description': 'lcmap_permanent_forest',
        'image': forest_sum_float,
        'maxPixels':1e13, 
        'scale' : 30
    }
    
task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [40]:
task.status()

{'state': 'RUNNING',
 'description': 'lcmap_permanent_forest',
 'creation_timestamp_ms': 1655856184706,
 'update_timestamp_ms': 1655856559442,
 'start_timestamp_ms': 1655856196847,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'U6GBBQJ3UZK2HA7YSGI6KRLZ',
 'name': 'projects/earthengine-legacy/operations/U6GBBQJ3UZK2HA7YSGI6KRLZ'}